## Step 4: Train and Evaluate QSVM (RUNS ON IBM'S QC)

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from qiskit_ibm_provider import IBMProvider
from qiskit.primitives import Sampler
import numpy as np

# Load IBM Quantum account
provider = IBMProvider(token="f73393d0b4e099ab95417406a2c31844acdfaaf9373a2e0a3366ea247a6fb275652ea4d3fcb4ba2de1318577d3bced2d09aeb9610f761eaddb0a983d6b033661")  # Replace with your actual API key

# List available backends
backends = provider.backends()
print("Available backends:", backends)

# Define backends (update with available backends)
qasm_backend = provider.get_backend('ibmq_qasm_simulator')  # Check the correct backend name
statevector_backend = provider.get_backend('ibmq_statevector_simulator')  # Check the correct backend name

# Define the feature map
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')

# Create a Sampler instance
sampler = Sampler(backend=qasm_backend)

# Define the quantum kernel using FidelityQuantumKernel
kernel = FidelityQuantumKernel(fidelity=sampler, feature_map=feature_map)

# Sample data and labels
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [1, 1], [2, 3], [6, 5], [8, 7]])
y = np.array([0, 1, 1, 1, 0, 0, 1, 1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define a kernel function for SVC using the quantum kernel
def kernel_function(X1, X2):
    # Evaluates the kernel matrix between X1 and X2
    return kernel.evaluate(x_vec=X1, y_vec=X2)

# Create and train the SVC model with the quantum kernel
svc_model = SVC(kernel=kernel_function)
svc_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svc_model.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"QSVC Model Accuracy: {accuracy:.4f}")
print(f"Predictions: {y_pred}")


## Step 5: Train and Evaluate QSVM with QPCA

In [68]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.primitives import Sampler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from qiskit_ibm_provider import IBMProvider
from qiskit_algorithms import VQE

# Load IBM Quantum account
provider = IBMProvider(token="f73393d0b4e099ab95417406a2c31844acdfaaf9373a2e0a3366ea247a6fb275652ea4d3fcb4ba2de1318577d3bced2d09aeb9610f761eaddb0a983d6b033661")  # Replace with your actual API key

# Define the feature map for quantum kernel
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement="linear")

# Define the backends
backend_qasm = provider.get_backend("ibmq_qasm_simulator")
backend_statevector = provider.get_backend("ibmq_statevector_simulator")

# Create a sampler and Fidelity instance
sampler = Sampler(backend=backend_qasm)
fidelity = FidelityQuantumKernel(sampler=sampler, feature_map=feature_map)

# Create the Quantum Kernel
quantum_kernel_qasm = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)
quantum_kernel_statevector = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

# Example training features and labels
train_features = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
train_labels = np.array([0, 1, 1, 0])

# Scale features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)

# Perform PCA on the classical data
pca = PCA(n_components=2)
train_features_pca = pca.fit_transform(train_features_scaled)

# Define and fit the QSVC model for qasm_simulator
qsvc_qasm = QSVC(quantum_kernel=quantum_kernel_qasm)
qsvc_qasm.fit(train_features_pca, train_labels)

# Evaluate the QSVC model for qasm_simulator
accuracy_qasm = qsvc_qasm.score(train_features_pca, train_labels)
print(f"QSVC Model Accuracy with QASM Simulator: {accuracy_qasm:.4f}")

# Define and fit the QSVC model for statevector_simulator
qsvc_statevector = QSVC(quantum_kernel=quantum_kernel_statevector)
qsvc_statevector.fit(train_features_pca, train_labels)

# Evaluate the QSVC model for statevector_simulator
accuracy_statevector = qsvc_statevector.score(train_features_pca, train_labels)
print(f"QSVC Model Accuracy with Statevector Simulator: {accuracy_statevector:.4f}")

# Optionally, make predictions
predictions_qasm = qsvc_qasm.predict(train_features_pca)
predictions_statevector = qsvc_statevector.predict(train_features_pca)

print("Predictions with QASM Simulator:", predictions_qasm)
print("Predictions with Statevector Simulator:", predictions_statevector)

# Optionally visualize the PCA results
plt.figure(figsize=(10, 5))
plt.scatter(train_features_pca[:, 0], train_features_pca[:, 1], c=train_labels, cmap='viridis')
plt.title("PCA of Training Features")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar(label='Labels')
plt.show()


RequestsApiError: '401 Client Error: Unauthorized for url: https://api.quantum.ibm.com/runtime/backends/?provider=ibm-q-wits%2Finternal%2Fwits-eie. {"errors":[{"code":1201,"message":"Invalid credentials.","solution":"Verify your credentials and try again.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"cr1hiac89r83dfear17g"}'